In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm

class Investment:
    def __init__(self, stage, outcome_distribution, ticket_size_mean, ticket_size_stdev):
        self.stage = stage
        self.outcome_distribution = outcome_distribution
        self.ticket_size = np.random.normal(ticket_size_mean, ticket_size_stdev)
        self.status = "active"
        self.value = 0

    def simulate_year(self):
        outcomes = list(self.outcome_distribution.keys())
        probabilities = [float(probability.value) for probability in self.outcome_distribution.values()]
        outcome = np.random.choice(outcomes, p=probabilities)
        if outcome == "writeoff":
            self.status = "writeoff"
            self.value = 0
        elif outcome == "exit":
            self.status = "exit"
            self.value = self.ticket_size * 3  # Assuming exit at 3 times the investment
        elif outcome == "same":
            self.status = "same"
            self.value = self.ticket_size  # Assuming value remains the same
        # Add more outcomes as needed



class Fund:
    def __init__(self, committed_capital, management_fee, lifetime, extension_years, carried_interest, hurdle_rate, capital_allocation, average_ticket_size, outcome_distributions, outcome_probabilities):
        self.committed_capital = committed_capital
        self.management_fee = management_fee
        self.lifetime = lifetime
        self.extension_years = extension_years
        self.carried_interest = carried_interest
        self.hurdle_rate = hurdle_rate
        self.capital_allocation = capital_allocation
        self.average_ticket_size = average_ticket_size
        self.outcome_distributions = outcome_distributions
        self.outcome_probabilities = outcome_probabilities
        self.investments = []
        self.cash_flow = 0
        self.returns = 0
        self.data = pd.DataFrame(columns=['Year', 'New Investments', 'Follow-on Investments', 'Portfolio Companies', 'Management Fees', 'Investments', 'Divestments', 'Distributions'])
        self.waterfall = pd.DataFrame(columns=['Year', 'LPs Principal', 'Preferred LPs Returns', 'GPs Catch Up', 'LPs Additional Profits', 'GPs Additional Profits'])

    def allocate_capital(self, year):
        # Allocate capital based on the capital_allocation percentages
        for stage, allocation_widget in self.capital_allocation.items():
            allocation = allocation_widget.value  # Retrieve the value of the FloatSlider widget
            mean_value = self.average_ticket_size[stage]['mean'].value  # Retrieve the value of the IntSlider widget
            stdev_value = self.average_ticket_size[stage]['stdev'].value  # Retrieve the value of the IntSlider widget
            num_investments = int(allocation * self.committed_capital / mean_value)
            for _ in range(num_investments):
                self.make_investment(stage, self.outcome_distributions[stage], mean_value, stdev_value)

    def make_investment(self, stage, outcome_distribution, ticket_size_mean, ticket_size_stdev):
        investment = Investment(stage, outcome_distribution, ticket_size_mean, ticket_size_stdev)
        self.investments.append(investment)

    # ... rest of the Fund class ...


    def simulate_year(self, year):
        new_investments = 0
        follow_on_investments = 0
        portfolio_companies = 0
        management_fees = self.committed_capital * self.management_fee
        investments = 0
        divestments = 0
        distributions = 0

        for investment in self.investments:
            if investment.status == "active":
                portfolio_companies += 1
                investment.simulate_year()
                if investment.status == "exit":
                    divestments += investment.value
                    distributions += investment.value
                elif investment.status == "writeoff":
                    divestments += investment.ticket_size
                investments += investment.ticket_size

        # ... continuation of Fund class ..
        self.data.loc[year] = [year, new_investments, follow_on_investments, portfolio_companies, management_fees, investments, divestments, distributions]

        # Update waterfall DataFrame
        lps_principal = min(self.committed_capital, distributions)
        distributions -= lps_principal
        preferred_lps_returns = min(self.committed_capital * self.hurdle_rate, distributions)
        distributions -= preferred_lps_returns
        gps_catch_up = min((self.committed_capital * self.hurdle_rate * self.carried_interest) / (1 - self.carried_interest), distributions)
        distributions -= gps_catch_up
        lps_additional_profits = distributions * (1 - self.carried_interest)
        gps_additional_profits = distributions * self.carried_interest

        self.waterfall.loc[year] = [year, lps_principal, preferred_lps_returns, gps_catch_up, lps_additional_profits, gps_additional_profits]

class Simulator:
    def __init__(self, fund):
        self.fund = fund

    def run_simulation(self):
        for year in range(self.fund.lifetime + self.fund.extension_years):
            self.fund.allocate_capital(year)
            self.fund.simulate_year(year)
        return self.fund.data, self.fund.waterfall


In [3]:
from ipywidgets import interact, fixed, FloatSlider, IntSlider, FloatText, IntText



# Define stages
stages = ["pre-seed", "seed", "post-seed", "series A"]

# Function to run simulation
def run_simulation(committed_capital, management_fee, lifetime, extension_years, carried_interest, hurdle_rate, capital_allocation, average_ticket_size, outcome_distributions, outcome_probabilities):
    fund = Fund(committed_capital, management_fee, lifetime, extension_years, carried_interest, hurdle_rate, capital_allocation, average_ticket_size, outcome_distributions, outcome_probabilities)
    simulator = Simulator(fund)
    data, waterfall = simulator.run_simulation()
    return data, waterfall

# Widgets for parameters
committed_capital = IntSlider(min=1000000, max=100000000, step=1000000, value=10000000, description="Committed Capital")
management_fee = FloatSlider(min=0, max=1, step=0.01, value=0.02, description="Management Fee")
lifetime = IntSlider(min=1, max=20, step=1, value=10, description="Lifetime")
extension_years = IntSlider(min=0, max=10, step=1, value=2, description="Extension Years")
carried_interest = FloatSlider(min=0, max=1, step=0.01, value=0.2, description="Carried Interest")
hurdle_rate = FloatSlider(min=0, max=1, step=0.01, value=0.08, description="Hurdle Rate")

# Widgets for parameters
capital_allocation = {stage: FloatSlider(min=0, max=1, step=0.01, value=0.25, description=f"Capital Allocation ({stage})") for stage in stages}
average_ticket_size = {stage: {"mean": IntSlider(min=100000, max=10000000, step=100000, value=1000000, description=f"Average Ticket Size Mean ({stage})"),
                               "stdev": IntSlider(min=10000, max=1000000, step=10000, value=200000, description=f"Average Ticket Size Stdev ({stage})")} for stage in stages}

outcome_distributions = {stage: {"writeoff": FloatText(value=0, description=f"Writeoff Multiplier ({stage})"),
                                 "liquidation": {"mean": FloatText(value=0.3, description=f"Liquidation Mean ({stage})"), "stdev": FloatText(value=0.2, description=f"Liquidation Stdev ({stage})")},
                                 "normal_exit": {"mean": FloatText(value=2.5, description=f"Normal Exit Mean ({stage})"), "stdev": FloatText(value=1, description=f"Normal Exit Stdev ({stage})")},
                                 "big_winner": {"mean": FloatText(value=7, description=f"Big Winner Mean ({stage})"), "stdev": FloatText(value=3, description=f"Big Winner Stdev ({stage})")}} for stage in stages}

outcome_probabilities = {stage: [FloatSlider(min=0, max=1, step=0.01, value=0.5, description=f"Writeoff Probability ({stage})"),
                                 FloatSlider(min=0, max=1, step=0.01, value=0.166, description=f"Liquidation Probability ({stage})"),
                                 FloatSlider(min=0, max=1, step=0.01, value=0.166, description=f"Normal Exit Probability ({stage})"),
                                 FloatSlider(min=0, max=1, step=0.01, value=0.166, description=f"Big Winner Probability ({stage})")] for stage in stages}

# Interactive function
@interact(committed_capital=committed_capital,
          management_fee=management_fee,
          lifetime=lifetime,
          extension_years=extension_years,
          carried_interest=carried_interest,
          hurdle_rate=hurdle_rate,
          capital_allocation=fixed(capital_allocation),
          average_ticket_size=fixed(average_ticket_size),
          outcome_distributions=fixed(outcome_distributions),
          outcome_probabilities=fixed(outcome_probabilities))

def run_simulation_widget(committed_capital, management_fee, lifetime, extension_years, carried_interest, hurdle_rate, capital_allocation, average_ticket_size, outcome_distributions, outcome_probabilities):
    fund = Fund(committed_capital, management_fee, lifetime, extension_years, carried_interest, hurdle_rate, capital_allocation, average_ticket_size, outcome_distributions, outcome_probabilities)
    simulator = Simulator(fund)
    data, waterfall = simulator.run_simulation()
    return data, waterfall



interactive(children=(IntSlider(value=10000000, description='Committed Capital', max=100000000, min=1000000, s…

<function __main__.run_simulation(committed_capital, management_fee, lifetime, extension_years, carried_interest, hurdle_rate, capital_allocation, average_ticket_size, outcome_distributions, outcome_probabilities)>